# Quantize Speech Recognition Models using NNCF PTQ API
This tutorial demonstrates how to use the NNCF (Neural Network Compression Framework) 8-bit quantization in post-training mode (without the fine-tuning pipeline) to optimize the speech recognition model, known as [Data2Vec](https://arxiv.org/abs/2202.03555) for the high-speed inference via OpenVINO™ Toolkit. This notebook uses a fine-tuned [data2vec-audio-base-960h](https://huggingface.co/facebook/data2vec-audio-base-960h) [PyTorch](https://pytorch.org/) model trained on the [LibriSpeech ASR corpus](https://www.openslr.org/12). The tutorial is designed to be extendable to custom models and datasets. It consists of the following steps:

- Download and prepare model.
- Define data loading and accuracy validation functionality.
- Prepare the model for quantization and quantize.
- Compare performance of the original and quantized models.
- Compare Accuracy of the Original and Quantized Models.

## Download and prepare model

data2vec is a framework for self-supervised representation learning for images, speech, and text as described in [data2vec: A General Framework for Self-supervised Learning in Speech, Vision and Language (Baevski et al., 2022)](https://ai.facebook.com/research/data2vec-a-general-framework-for-self-supervised-learning-in-speech-vision-and-language). The algorithm uses the same learning mechanism for different modalities.

![pre-trained pipeline](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/data2vec.png)

In our case, we will use `data2vec-audio-base-960h` model, which was finetuned on 960 hours of audio from LibriSpeech Automatic Speech Recognition corpus and distributed as part of HuggingFace transformers.

### Obtain Pytorch model representation

For instantiating PyTorch model class, we should use `Data2VecAudioForCTC.from_pretrained` method with providing model ID for downloading from HuggingFace hub. Model weights and configuration files will be downloaded automatically in first time usage.
Keep in mind that downloading the files can take several minutes and depends on your internet connection.

Additionally, we can create processor class which is responsible for model specific pre- and post-processing steps.

In [ ]:
!pip install -q soundfile librosa

In [ ]:
from transformers import Wav2Vec2Processor, Data2VecAudioForCTC

processor = Wav2Vec2Processor.from_pretrained("facebook/data2vec-audio-base-960h")
model = Data2VecAudioForCTC.from_pretrained("facebook/data2vec-audio-base-960h")

### Convert model to OpenVINO Intermediate Representation



In [ ]:
from pathlib import Path
# Set model directory
MODEL_DIR = Path("model")
MODEL_DIR.mkdir(exist_ok=True)

In [ ]:
from openvino.tools import mo
from openvino.runtime import serialize, Core
import torch

core = Core()

BATCH_SIZE = 1
MAX_SEQ_LENGTH = 30480


def export_model_to_onnx(model, path):
    # switch model to evaluation mode
    model.eval()
    # disallow gradient propagation for reducing memory during export
    with torch.no_grad():
        # define dummy input with specific shape
        default_input = torch.zeros([1, MAX_SEQ_LENGTH], dtype=torch.float)
        inputs = {
            "inputs": default_input
        }

        # define names for dynamic dimentions
        symbolic_names = {0: "batch_size", 1: "sequence_len"}
        # export model
        torch.onnx.export(
            model,
            (inputs["inputs"]),
            path,
            opset_version=11,
            input_names=["inputs"],
            output_names=["logits"],
            dynamic_axes={
                "inputs": symbolic_names,
                "logits": symbolic_names,
            },
        )
        print("ONNX model saved to {}".format(path))


onnx_model_path = MODEL_DIR / "data2vec-audo-base.onnx"
ir_model_path = onnx_model_path.with_suffix('.xml')

if not ir_model_path.exists():
    if not onnx_model_path.exists():
        export_model_to_onnx(model, onnx_model_path)
    ov_model = mo.convert_model(onnx_model_path, compress_to_fp16=True)
    serialize(ov_model, str(ir_model_path))
    print("IR model saved to {}".format(ir_model_path))
else:
    print("Read IR model from {}".format(ir_model_path))
    ov_model = core.read_model(ir_model_path)


### Prepare inference data

For demonstration purposes, we will use short dummy version of librispeach dataset - `patrickvonplaten/librispeech_asr_dummy` to speed up model evaluation. Model accuracy can be different from reported in the paper. For reproducing original accuracy, use `librispeech_asr` dataset.

In [ ]:
!pip install -q datasets "torchmetrics>=0.11.0"

In [ ]:
from datasets import load_dataset

ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")


# define preprocessing function for converting audio to input values for model
def map_to_input(batch):
    preprocessed_signal = processor(batch["audio"]["array"], return_tensors="pt", padding="longest", sampling_rate=batch['audio']['sampling_rate'])
    input_values = preprocessed_signal.input_values
    batch['input_values'] = input_values
    return batch


# apply preprocessing function to dataset and remove audio column, to save memory as we do not need it anymore
dataset = ds.map(map_to_input, batched=False, remove_columns=["audio"])

test_sample = ds[0]["audio"]

## Check model inference result

The code below is used for running model inference on a single sample from the dataset. It contains the following steps:

* Get the input_values tensor as model input.
* Run model inference and obtain logits.
* Find logits ids with highest probability, using argmax.
* Decode predicted token ids, using processor.

For reference, see the same function provided for OpenVINO model.

In [ ]:
import numpy as np


# inference function for pytorch
def torch_infer(model, sample):
    logits = model(torch.Tensor(sample['input_values'])).logits
    # take argmax and decode
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription


# inference function for openvino
def ov_infer(model, sample):
    output = model.output(0)
    logits = model(np.array(sample['input_values']))[output]
    predicted_ids = np.argmax(logits, axis=-1)
    transcription = processor.batch_decode(torch.from_numpy(predicted_ids))
    return transcription

In [ ]:
core = Core()

pt_transcription = torch_infer(model, dataset[0])
compiled_model = core.compile_model(ov_model)
ov_transcription = ov_infer(compiled_model, dataset[0])

In [ ]:
import IPython.display as ipd

print(f"[Reference]:     {dataset[0]['text']}")
print(f"[PyTorch]:       {pt_transcription[0]}")
print(f"[OpenVINO FP16]: {ov_transcription[0]}")
ipd.Audio(test_sample["array"], rate=16000)

## Validate model accuracy on dataset

For model accuracy evaluation, [Word Error Rate](https://en.wikipedia.org/wiki/Word_error_rate) metric can be used. Word Error Rate or WER is the ratio of errors in a transcript to the total words spoken. A lower WER in speech-to-text means better accuracy in recognizing speech.

For WER calculation, we will use [torchmetrics](https://torchmetrics.readthedocs.io/en/stable/text/word_error_rate.html) library.

In [ ]:
from torchmetrics import WordErrorRate
from tqdm.notebook import tqdm


def compute_wer(dataset, model, infer_fn):
    wer = WordErrorRate()
    for sample in tqdm(dataset):
        # run infer function on sample
        transcription = infer_fn(model, sample)
        # update metric on sample result
        wer.update(transcription, [sample['text']])
    # finalize metric calculation
    result = wer.compute()
    return result

In [ ]:
pt_result = compute_wer(dataset, model, torch_infer)
ov_result = compute_wer(dataset, compiled_model, ov_infer)

In [ ]:
print(f'[PyTorch]   Word Error Rate: {pt_result:.4f}')
print(f'[OpenVino]  Word Error Rate: {ov_result:.4f}')

## Quantization

[NNCF](https://github.com/openvinotoolkit/nncf) provides a suite of advanced algorithms for Neural Networks inference optimization in OpenVINO with minimal accuracy drop.

Create a quantized model from the pre-trained `FP16` model and the calibration dataset. The optimization process contains the following steps:
    1. Create a Dataset for quantization.
    2. Run `nncf.quantize` for getting an optimized model. The `nncf.quantize` function provides an interface for model quantization. It requires an instance of the OpenVINO Model and quantization dataset.
Optionally, some additional parameters for the configuration quantization process (number of samples for quantization, preset, ignored scope, etc.) can be provided. For more accurate results, we should keep the operation in the postprocessing subgraph in floating point precision, using the `ignored_scope` parameter. `advanced_parameters` can be used to specify advanced quantization parameters for fine-tuning the quantization algorithm. In this tutorial we pass range estimator parameters for activations. For more information see [Tune quantization parameters](https://docs.openvino.ai/2023.0/basic_quantization_flow.html#tune-quantization-parameters).
    3. Serialize OpenVINO IR model using `openvino.runtime.serialize` function.

In [ ]:
import nncf
from nncf.quantization.advanced_parameters import AdvancedQuantizationParameters
from nncf.quantization.range_estimator import AggregatorType
from nncf.quantization.range_estimator import RangeEstimatorParameters
from nncf.quantization.range_estimator import StatisticsCollectorParameters
from nncf.quantization.range_estimator import StatisticsType
from nncf.parameters import ModelType


def transform_fn(data_item):
    """
    Extract the model's input from the data item.
    The data item here is the data item that is returned from the data source per iteration.
    This function should be passed when the data item cannot be used as model's input.
    """
    return np.array(data_item["input_values"])


calibration_dataset = nncf.Dataset(dataset, transform_fn)


quantized_model = nncf.quantize(
    ov_model,
    calibration_dataset,
    model_type=ModelType.TRANSFORMER,  # specify additional transformer patterns in the model
    subset_size=len(dataset),
    ignored_scope=nncf.IgnoredScope(
        names=[
            "/data2vec_audio/encoder/layers.3/feed_forward/intermediate_dense/MatMul",
            "/data2vec_audio/feature_extractor/conv_layers.2/conv/Conv",
            "/data2vec_audio/encoder/layers.3/Add_1",
            "/data2vec_audio/encoder/layers.2/feed_forward/intermediate_dense/MatMul",
            "/data2vec_audio/feature_extractor/conv_layers.0/conv/Conv",
            "/data2vec_audio/encoder/layers.4/Add_1",
            "/data2vec_audio/encoder/layers.4/feed_forward/intermediate_dense/MatMul",
            "/data2vec_audio/encoder/layers.4/final_layer_norm/Div",
            "/data2vec_audio/encoder/layers.4/feed_forward/output_dense/MatMul",
            "/data2vec_audio/encoder/layers.8/attention/MatMul_1",
            "/data2vec_audio/feature_extractor/conv_layers.1/conv/Conv",
            "/data2vec_audio/encoder/layers.2/Add_1",
            "/data2vec_audio/feature_extractor/conv_layers.0/layer_norm/Div",
            "/data2vec_audio/encoder/layers.1/feed_forward/intermediate_dense/MatMul",
            "/data2vec_audio/encoder/layers.1/Add_1",
            "/data2vec_audio/feature_extractor/conv_layers.3/layer_norm/Div"
        ],
    ),
    advanced_parameters=AdvancedQuantizationParameters(
        activations_range_estimator_params=RangeEstimatorParameters(
            min=StatisticsCollectorParameters(
                statistics_type=StatisticsType.MIN,
                aggregator_type=AggregatorType.MIN
            ),
            max=StatisticsCollectorParameters(
                statistics_type=StatisticsType.QUANTILE,
                aggregator_type=AggregatorType.MEAN,
                quantile_outlier_prob=0.0001
            ),
        )
    )
)

After quantization is finished, compressed model representation can be saved using `serialize` function.

In [ ]:
MODEL_NAME = 'quantized_data2vec_base'
quantized_model_path = Path(f"{MODEL_NAME}_openvino_model/{MODEL_NAME}_quantized.xml")
serialize(quantized_model, str(quantized_model_path))

## Check INT8 model inference result

`INT8` model is the same in usage like the original one. We need to read it, using the `core.read_model` method and load on the device, using `core.compile_model`. After that, we can reuse the same `ov_infer` function for getting model inference result on test sample.

In [ ]:
int8_compiled_model = core.compile_model(quantized_model)

In [ ]:
transcription = ov_infer(int8_compiled_model, dataset[0])
print(f"[Reference]:     {dataset[0]['text']}")
print(f"[OpenVINO INT8]: {transcription[0]}")
ipd.Audio(test_sample["array"], rate=16000)

## Compare Performance of the Original and Quantized Models
[Benchmark Tool](https://docs.openvino.ai/latest/openvino_inference_engine_tools_benchmark_tool_README.html) is used to measure the inference performance of the `FP16` and `INT8` models.

> **NOTE**: For more accurate performance, it is recommended to run `benchmark_app` in a terminal/command prompt after closing other applications. Run `benchmark_app -m model.xml -d CPU` to benchmark async inference on CPU for one minute. Change `CPU` to `GPU` to benchmark on GPU. Run `benchmark_app --help` to see an overview of all command-line options.

In [ ]:
# Inference FP16 model (OpenVINO IR)
! benchmark_app -m $ir_model_path -shape [1,30480] -d CPU -api async -t 15

In [ ]:
# Inference INT8 model (OpenVINO IR)
! benchmark_app -m $quantized_model_path -shape [1,30480] -d CPU -api async -t 15

## Compare Accuracy of the Original and Quantized Models

Finally, calculate WER metric for the `INT8` model representation and compare it with the `FP16` result.

In [ ]:
int8_ov_result = compute_wer(dataset, int8_compiled_model, ov_infer)
print(f'[OpenVino FP16] Word Error Rate: {ov_result:.4}')
print(f'[OpenVino INT8] Word Error Rate: {int8_ov_result:.4f}')